In [1]:
import pandas as pd
import folium
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "plotly_white"
metro_data = pd.read_csv("Delhi Metro Network new.csv")
#print(metro_data.head())

/Users/gaurav/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
# Color scheme
line_colors = {
    'Red line': 'red',
    'Blue line': 'blue',
    'Yellow line': 'beige',
    'Green line': 'green',
    'Voilet line': 'purple',
    'Pink line': 'pink',
    'Magenta line': 'darkred',
    'Orange line': 'orange',
    'Rapid Metro': 'cadetblue',
    'Aqua line': 'black',
    'Green line branch': 'lightgreen',
    'Blue line branch': 'lightblue',
    'Gray line': 'lightgray'
}

# Create map centered on Delhi
delhi_map_with_line_tooltip = folium.Map(location=[28.7041, 77.1025], zoom_start=11)

# Add stations
for _, row in metro_data.iterrows():
    line = row['Line']
    color = line_colors.get(line, 'black')
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['Station Name'],
        tooltip=f"{row['Station Name']}, {line}",
        icon=folium.Icon(color=color)
    ).add_to(delhi_map_with_line_tooltip)

# Save and open
delhi_map_with_line_tooltip.save("delhi_metro_map.html")

import webbrowser
webbrowser.open("delhi_metro_map.html")

True

In [3]:
# Convert to datetime
metro_data['Opening Date'] = pd.to_datetime(metro_data['Opening Date'], errors='coerce')

# Drop invalid dates
metro_data = metro_data.dropna(subset=['Opening Date'])

# Extract year
metro_data['Opening Year'] = metro_data['Opening Date'].dt.year

# Count number of stations per year
stations_per_year = metro_data['Opening Year'].value_counts().sort_index()
stations_per_year_df = stations_per_year.reset_index()
stations_per_year_df.columns = ['Year', 'Number of Stations']

# Plotting
fig = px.bar(stations_per_year_df, x ='Year', y ='Number of Stations',
             title="Number of Metro Stations Opened Each Year in Delhi",
             labels={'Year': 'Year', 'Number of Stations': 'Number of Stations Opened'})

fig.update_layout(xaxis_tickangle=-45, xaxis=dict(tickmode='linear'),
                  yaxis=dict(title='Number of Stations Opened'),
                  xaxis_title="Year")

fig.show()

/var/folders/d8/kcg69jfx7xggbl3rc1p2gdcr0000gn/T/ipykernel_14079/1760375022.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  metro_data['Opening Date'] = pd.to_datetime(metro_data['Opening Date'], errors='coerce')


In [4]:
stations_per_line = metro_data['Line'].value_counts()

# calculating the total distance of each metro line (max distance from start)
total_distance_per_line = metro_data.groupby('Line')['Distance from Start (km)'].max()

avg_distance_per_line = total_distance_per_line / (stations_per_line - 1)


# Combine into one DataFrame using proper alignment
line_analysis = pd.concat([stations_per_line, total_distance_per_line, avg_distance_per_line], axis=1)
line_analysis.columns = ['Number of Stations', 'Total Distance (km)', 'Average Distance Between Stations (km)']

# Sort by Number of Stations
line_analysis = line_analysis.sort_values(by='Number of Stations', ascending=False).reset_index()
line_analysis.rename(columns={'index': 'Line'}, inplace=True)

print(line_analysis)

                 Line  Number of Stations  Total Distance (km)  \
0           Blue line                  49                 52.7   
1           Pink line                  38                 52.6   
2         Yellow line                  37                 45.7   
3         Voilet line                  34                 43.5   
4            Red line                  29                 32.7   
5        Magenta line                  26                 35.6   
6          Green line                  22                 24.8   
7           Aqua line                  21                 27.1   
8         Rapid Metro                  11                 10.0   
9    Blue line branch                   8                  8.1   
10        Orange line                   7                 22.7   
11          Grey line                   4                  5.2   
12  Green line branch                   3                  2.1   

    Average Distance Between Stations (km)  
0                             

In [5]:
# creating subplots
fig = make_subplots(rows=1, cols=2, subplot_titles=('Number of Stations Per Metro Line',
                                                    'Average Distance Between Stations Per Metro Line'),
                    horizontal_spacing=0.2)

# plot for Number of Stations per Line
fig.add_trace(
    go.Bar(y=line_analysis['Line'], x=line_analysis['Number of Stations'],
           orientation='h', name='Number of Stations', marker_color='crimson'),
    row=1, col=1
)

# plot for Average Distance Between Stations
fig.add_trace(
    go.Bar(y=line_analysis['Line'], x=line_analysis['Average Distance Between Stations (km)'],
           orientation='h', name='Average Distance (km)', marker_color='navy'),
    row=1, col=2
)

# update xaxis properties
fig.update_xaxes(title_text="Number of Stations", row=1, col=1)
fig.update_xaxes(title_text="Average Distance Between Stations (km)", row=1, col=2)

# update yaxis properties
fig.update_yaxes(title_text="Metro Line", row=1, col=1)
fig.update_yaxes(title_text="", row=1, col=2)

# update layout
fig.update_layout(height=600, width=1200, title_text="Metro Line Analysis", template="plotly_white")

fig.show()

In [6]:
layout_counts = metro_data['Station Layout'].value_counts()

# creating the bar plot using Plotly
fig = px.bar(x=layout_counts.index, y=layout_counts.values,
             labels={'x': 'Station Layout', 'y': 'Number of Stations'},
             title='Distribution of Delhi Metro Station Layouts',
             color=layout_counts.index,
             color_continuous_scale='pastel')

# updating layout for better presentation
fig.update_layout(xaxis_title="Station Layout",
                  yaxis_title="Number of Stations",
                  coloraxis_showscale=False,
                  template="plotly_white")

fig.show()